In [3]:
import ccxt
from datetime import datetime
import pandas as pd
from sympy import Limit
import numpy as np
import time



## Fetch Ohlcv

In [9]:


def fetchData(exchange, symbol, timeframe, since="26-04-2022", limit=100):
    exchange_list = [
        'aax', 
        'ascendex', 
        'bequant', 
        'bibox',
        'bigone',
        'binance',
        'binancecoinm',
        'binanceus',
        'binanceusdm', 
        'bit2c', 
        'bitbank',
        'bitbay',
        'bitbns',
        'bitcoincom', 
        'bitfinex',
        'bitfinex2',
        'bitflyer', 
        'bitforex', 
        'bitget', 
        'bithumb', 
        'bitmart',
        'bitmex',
        'bitopro',
        'bitpanda', 
        'bitrue', 
        'bitso',
        'bitstamp', 
        'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cdax', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'eqonex', 'exmo', 'flowbtc', 'fmfwio', 'ftx', 'ftxus', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 'okex5', 'okx', 'paymium', 'phemex', 'poloniex', 'probit', 'qtrade', 'ripio', 'stex', 'therock', 'tidebit', 'tidex', 'timex', 'upbit', 'vcc', 'wavesexchange', 'wazirx', 'whitebit', 'woo', 'xena', 'yobit', 'zaif', 'zb', 'zipmex', 'zonda']
    try:
        exchange = getattr(ccxt, exchange)()
    except AttributeError:
        print("-" * 36, " ERROR ", "-" * 35)
        print(
            'Exchange "{}" not found. Please check the exchange is supported.'.format(
                exchange
            )
        )
        print("Supported exchanges are:")
        print(exchange_list)
        print("-" * 80)
        quit()
    data = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
    header = ["Timestamp", "open", "high", "low", "close", "volume"]
    df = pd.DataFrame(data, columns=header)

    df.Timestamp = (
        df.Timestamp / 1000
    ) 
    df["timestamp"] = pd.to_datetime(df.Timestamp, unit="s")

    df = df[["timestamp", "open", "high", "low", "close", "volume"]]

    df["open"] = pd.to_numeric(df["open"])
    df["high"] = pd.to_numeric(df["high"])
    df["low"] = pd.to_numeric(df["low"])
    df["close"] = pd.to_numeric(df["close"])
    df["volume"] = pd.to_numeric(df["volume"])

    return df

In [10]:
print(fetchData("kraken","BTC/USDT","1m",30))

             timestamp     open     high      low    close    volume
0  2022-04-28 08:12:00  39406.3  39406.3  39390.0  39390.0  0.196870
1  2022-04-28 08:13:00  39438.9  39438.9  39438.9  39438.9  0.001710
2  2022-04-28 08:14:00  39480.0  39490.3  39480.0  39490.3  0.200534
3  2022-04-28 08:15:00  39490.3  39490.3  39490.3  39490.3  0.000000
4  2022-04-28 08:16:00  39490.3  39490.3  39490.3  39490.3  0.000000
..                 ...      ...      ...      ...      ...       ...
95 2022-04-28 09:47:00  39795.1  39795.1  39789.3  39789.3  0.031457
96 2022-04-28 09:48:00  39806.8  39808.3  39806.8  39808.3  0.062812
97 2022-04-28 09:49:00  39808.3  39808.3  39808.3  39808.3  0.000000
98 2022-04-28 09:50:00  39805.5  39805.5  39805.5  39805.5  0.018780
99 2022-04-28 09:51:00  39805.5  39805.5  39805.5  39805.5  0.000000

[100 rows x 6 columns]


## Funciones-Fecthc Order Book para los distintos apis

In [444]:
def tim():
    t=time.localtime(time.time())
    return str(t[0])+"-"+str(t[1])+"-"+str(t[2])
tim()

'2022-4-28'

In [445]:
def fetch_order_book_bitso(symbol,limit):
    exchange = ccxt.bitso()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk


In [446]:
def fetch_order_book_ascendex(symbol,limit):
    exchange = ccxt.ascendex()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk


In [447]:
def fetch_order_book_bitbay(symbol,limit):
    exchange = ccxt.bitbay()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk

In [448]:
def fetch_order_book_bitbns(symbol,limit):
    exchange = ccxt.bitbns()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_time=bi_btc_ob["datetime"]
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk["Timestamp"]=bi_btc_ob_time
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Spread"]=(dfk.Bid - dfk.Ask)
    
    
                       
    return dfk